# Анализ производительности радиорелейной сети

В этом блокноте представлены результаты исследования производительности многошаговой радиорелейной сети с помощью имитационного моделирования и аналитических моделей сетей массового обслуживания с узлами MAP/PH/1/N.

# 1. Подготовка

Прежде, чем перейти к проведению эксперимента, подключим основные модули и определим переменные окружения, которые будут управлять экспериментом.

In [5]:
# Если что-то меняется в коде, мы хотим это сразу видеть здесь
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Подключаем основные библиотеки для работы с данными, рисования графиков
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import itertools
import re
from tqdm import tqdm_notebook, tnrange

In [8]:
# Настраиваем matplotlib
%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})

Чтобы избежать длительных расчетов при работе с блокнотом, будем использовать два параметра:

- `RUN_MODE: str`: `"draft"` или `"production"`, определяет длительность моделирования и, соответственно, точность результатов. Для `RUN_MODE = "production"` точность гораздо выше, но требуется больше времени. Конкретные значения моделируемого времени определяются далее.
- `USE_CACHED: bool`: использовать или нет полученные ранее результаты из файлов. Если эксперимент был проведен ранее, и его результаты хранятся в файле, то будут использоваться они. Если файла нет, или значение `USE_CACHED = True`, то эксперимент будет проведен заново, а результаты сохранены в файл.

In [9]:
RUN_MODE = "draft"
USE_CACHED = True

Определим основные параметры модели: длины заголовков, скорости трафика и пр.

In [11]:
PHY_HEADER = 128   # длина заголовка физического уровня в битах: 128 бита
MAC_HEADER = 272   # длина заголовка канального уровня в битах:  272 бита
IP_HEADER = 160    # длина заголовка сетевого уровня в битах:   160 бит
PREAMBLE = 0       # преамбулу не учитываем

# Для удобсатва, вводим общую длину всех заголовков:
ALL_HEADERS_SIZE = PHY_HEADER + MAC_HEADER + IP_HEADER

IFS = 128e-6    # межкадровый интервал: 128 мкс
BITRATE = 1e6   # битовая скорость: 1 мегабит/с